In [1]:
# smallest possible test case for SQL query using jupyter notebook cell magic commands
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

## Claude and GPT-4 query

```
Please write a SQL query in duckdb with jupysql in a jupyter notebook to extract the column names from this csv file from the URL to a CSV

https://data.payless.health/hospital\_price\_transparency%2F010029\_636000526\_eastalabama\_standardcharges\_updatedsheet.csv
```

In [6]:
%%sql
SELECT * FROM read_csv_auto('https://data.payless.health/hospital_price_transparency%2F010029_636000526_eastalabama_standardcharges_updatedsheet.csv', header=true) 
LIMIT 1

(duckdb.InvalidInputException) Invalid Input Error: Error in file "https://data.payless.health/hospital_price_transparency%2F010029_636000526_eastalabama_standardcharges_updatedsheet.csv" at line 18569 in column ""column01"": Invalid unicode (byte sequence mismatch) detected in CSV file. Parser options:
  file=https://data.payless.health/hospital_price_transparency%2F010029_636000526_eastalabama_standardcharges_updatedsheet.csv
  delimiter=',' (auto detected)
  quote='"' (auto detected)
  escape='' (auto detected)
  header=1
  sample_size=20480
  ignore_errors=0
  all_varchar=0
[SQL: SELECT * FROM read_csv_auto('https://data.payless.health/hospital_price_transparency%2F010029_636000526_eastalabama_standardcharges_updatedsheet.csv', header=true) 
LIMIT 1]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [7]:
import pandas as pd



In [12]:
## download a csv file from a url using the requests library
import requests
url = 'https://data.payless.health/hospital_price_transparency%2F010029_636000526_eastalabama_standardcharges_updatedsheet.csv'
r = requests.get(url, allow_redirects=True)

# save the file to disk
open('eastalabama_standardcharges_updatedsheet.csv', 'wb').write(r.content)


16689693

In [13]:
import chardet 

chardet.detect(r.content)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}

In [14]:
import pandas as pd

# Read the CSV file with a specific encoding, such as 'ISO-8859-1' or 'utf-8'
df = pd.read_csv('eastalabama_standardcharges_updatedsheet.csv', encoding='Windows-1252')

# Write the DataFrame to a new CSV file with UTF-8 encoding
df.to_csv('/tmp/out.csv', index=False, encoding='utf-8')

/var/folders/2c/hmpw9qgd7fd3x0ms6548_f880000gn/T/ipykernel_99355/398947727.py:4: DtypeWarning: Columns (10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('eastalabama_standardcharges_updatedsheet.csv', encoding='Windows-1252')


In [15]:
!head /tmp/out.csv

Table 1: Standard charges,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
Last Updated 03/10/2022,,,,,,,,,,,,,,,,,,
Charge Code,Description,CPT/HCPCS,Revenue Code,CPT-4 Modifier,Gross Charge/Unit,Cash Price,Minimum Allowable,Maximum Allowable,"Alabama Crime Victims
(IP)",Beechstreet PHCS (OP),Coventry First Health (IP/OP),"GEHA
(OP)","MCM Maxcare
(IP/OP)","Mega Life
(IP/OP)","Multiplan PHCS
(OP)","TRPN 
(IP/OP)","Workers Comp 
(IP)","Workers Comp 


In [18]:
%%sql
SELECT * FROM read_csv_auto('/tmp/out.csv', header=true) 
LIMIT 10

,Table 1: Standard charges,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,Last Updated 03/10/2022,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Charge Code,Description,CPT/HCPCS,Revenue Code,CPT-4 Modifier,Gross Charge/Unit,Cash Price,Minimum Allowable,Maximum Allowable,Alabama Crime Victims\n(IP),Beechstreet PHCS (OP),Coventry First Health (IP/OP),GEHA\n(OP),MCM Maxcare\n(IP/OP),Mega Life\n(IP/OP),Multiplan PHCS\n(OP),TRPN \n(IP/OP),Workers Comp \n(IP),Workers Comp \n(OP)
2,5318710,ROOM/BED: Psych Private Room,None,114,None,"$1,235.00",$679.25,$617.50,"$1,111.50",$617.50,None,"$1,049.75",None,$988.00,$988.00,None,"$1,111.50",$741.00,None
3,5252283,ROOM/BED: IRF,None,118,None,"$1,235.00",$679.25,$617.50,"$1,111.50",$617.50,None,"$1,049.75",None,$988.00,$988.00,None,"$1,111.50",$741.00,None
4,5240485,ROOM/BED: Private Room,None,120,None,"$1,235.00",$679.25,$617.50,"$1,111.50",$617.50,None,"$1,049.75",None,$988.00,$988.00,None,"$1,111.50",$741.00,None
5,5240491,ROOM/BED: Long Term Adolescent,None,124,None,"$1,235.00",$679.25,$617.50,"$1,111.50",$617.50,None,"$1,049.75",None,$988.00,$988.00,None,"$1,111.50",$741.00,None
6,5240484,ROOM/BED: Nursery,None,171,None,"$1,235.00",$679.25,$617.50,"$1,111.50",$617.50,None,"$1,049.75",None,$988.00,$988.00,None,"$1,111.50",$741.00,None
7,5240489,ROOM/BED: Spec Care Nsy,None,172,None,"$1,852.00","$1,018.60",$926.00,"$1,666.80",$926.00,None,"$1,574.20",None,"$1,481.60","$1,481.60",None,"$1,666.80","$1,111.20",None
8,5199309,NBS NURSERY LEVEL III,None,173,None,"$1,852.00","$1,018.60",$926.00,"$1,666.80",$926.00,None,"$1,574.20",None,"$1,481.60","$1,481.60",None,"$1,666.80","$1,111.20",None
9,5199308,NBS ICU NURSERY,None,174,None,"$1,852.00","$1,018.60",$926.00,"$1,666.80",$926.00,None,"$1,574.20",None,"$1,481.60","$1,481.60",None,"$1,666.80","$1,111.20",None


In [19]:
df = _

In [ ]:
for column in df.columns:
    if df[column].isin(['CCN']).any():
        return index of row